# Get Tweets

This script extracts all the tweets with hashtag #covid-19 related to the day before today (yesterday) and saves them into a .csv file.
We use the `tweepy` library, which can be installed with the command `pip install tweepy`.

Firstly, we import the configuration file, called `config.py`, which is located in the same directory of this script.

In [113]:
from config import *
import tweepy
import datetime

We setup the connection to our Twitter App by using the `OAuthHandler()` class and its `access_token()` function. Then we call the Twitter API through the `API()` function.

In [114]:
auth = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True)

Now we setup dates. We need to setup today and yesterday.

In [115]:
today = datetime.date.today()
yesterday= today - datetime.timedelta(days=1)
today, yesterday

(datetime.date(2021, 3, 19), datetime.date(2021, 3, 18))

We search for tweets on Twitter by using the `Cursor()` function. 
We pass the `api.search` parameter to the cursor, as well as the query string, which is specified through the `q` parameter of the cursor.
The query string can receive many parameters, such as the following (not mandatory) ones:
* `from:` - to specify a specific Twitter user profile
* `since:` - to specify the beginning date of search
* `until:` - to specify the ending date of search
The cursor can also receive other parameters, such as the language and the `tweet_mode`. If `tweet_mode='extended'`, all the text of the tweet is returned, otherwise only the first 140 characters.

In [116]:
tweets_list = tweepy.Cursor(api.search, q="from:Agenzia_Ansa #Covid-19 since:" + str(yesterday)+ " until:" + str(today),tweet_mode='extended', lang='it').items()

Now we loop across the `tweets_list`, and, for each tweet, we extract the text, the creation date, the number of retweets and the favourite count. We store every tweet into a list, called `output`.

In [117]:
output = []
for tweet in tweets_list:
    text = tweet._json["full_text"]
    print(text)
    favourite_count = tweet.favorite_count
    retweet_count = tweet.retweet_count
    created_at = tweet.created_at
    
    line = {'text' : text, 'favourite_count' : favourite_count, 'retweet_count' : retweet_count, 'created_at' : created_at}
    output.append(line)

"Il numero di persone che muoiono a causa del Covid-19 in Europa è più alto ora rispetto allo scorso anno in questo periodo, il che riflette la vasta diffusione del virus". Lo ha detto il direttore dell'Oms Europa, Hansk Kluge #ANSA https://t.co/LNKv7ug7M4
Bandiere a mezz'asta per i palazzi delle istituzioni, gli edifici pubblici e i monumenti in Italia, in occasione della prima Giornata nazionale in memoria delle vittime della pandemia di Covid-19.  #ANSA #Giornatanazionale https://t.co/sAdXaQqIWg https://t.co/1zWtdMBonn
A Bergamo la cerimonia per la prima Giornata nazionale in memoria delle vittime della pandemia di Covid-19. Il premier Mario Draghi: "Lo Stato c'è e ci sarà, non possiamo abbracciarci ma dobbiamo sentirci tutti più uniti" #ANSA #GiornataNazionale https://t.co/FONQDugzh5 https://t.co/oBHaW2vdIn


Finally, we convert the `output` list to a `pandas DataFrame` and we store results.

In [118]:
import pandas as pd

df = pd.DataFrame(output)
#df.to_csv('output.csv', mode='a', header='False')
df.to_csv('output.csv')

In [119]:
df.head()

,text,favourite_count,retweet_count,created_at
0,"""Il numero di persone che muoiono a causa del ...",46,27,2021-03-18 12:24:22
1,Bandiere a mezz'asta per i palazzi delle istit...,34,14,2021-03-18 12:09:46
2,A Bergamo la cerimonia per la prima Giornata n...,100,28,2021-03-18 11:48:28
